In [1]:
import pandas as pd

In [2]:
import evaluate
from seqeval.metrics import (
    recall_score, 
    precision_score, 
    classification_report, 
    f1_score
)

/home/ionov/anaconda3/envs/kaggle/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_json('/archive/ionov/dataset/train.json')

In [4]:
df.head(3)

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."


In [5]:
predictions = pd.read_csv('presidio.csv', index_col=0)

In [6]:
predictions

,document,token,label
row_id,,,
0,7,9,B-NAME_STUDENT
1,7,10,I-NAME_STUDENT
2,7,52,B-NAME_STUDENT
3,7,53,I-NAME_STUDENT
4,7,55,B-NAME_STUDENT
...,...,...,...
11919,22664,284,B-NAME_STUDENT
11920,22664,285,I-NAME_STUDENT
11921,22677,4,B-URL_PERSONAL


In [7]:
predicted_labels = {doc_id: ['O'] * len(tokens) for doc_id, tokens in df.groupby('document')['tokens'].first().items()}

In [8]:
for index, row in predictions.iterrows():
    if row['document'] in predicted_labels and row['token'] < len(predicted_labels[row['document']]):
        predicted_labels[row['document']][row['token']] = row['label']

In [9]:
predicted_labels_list = []
for doc_id in df['document']:
    predicted_labels_list.append(predicted_labels[doc_id])

In [10]:
len(predicted_labels_list)

6807

In [11]:
len(df['labels'])

6807

In [12]:
seqeval_metrics = evaluate.load("seqeval")

def f5_score(precision, recall):
    return (1 + 5*5) * recall * precision / (5*5*precision + recall + 1e-100)

def compute_metrics_from_labels(labels, predictions):
    results = seqeval_metrics.compute(predictions=predictions, references=labels)
    for label, scores in results.items():
        if "overall" not in label:
            precision = scores["precision"]
            recall = scores["recall"]
            results[label]["f5_score"] = f5_score(precision, recall)
    precision = results["overall_precision"]
    recall = results["overall_recall"]
    results["overall_f5_score"] = f5_score(precision, recall)

    return results

In [13]:
metrics = compute_metrics_from_labels(df['labels'], predicted_labels_list)

/home/ionov/anaconda3/envs/kaggle/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
metrics

{'EMAIL': {'precision': 0.9743589743589743,
  'recall': 0.9743589743589743,
  'f1': 0.9743589743589743,
  'number': 39,
  'f5_score': 0.9743589743589743},
 'ID_NUM': {'precision': 0.0,
  'recall': 0.0,
  'f1': 0.0,
  'number': 78,
  'f5_score': 0.0},
 'NAME_STUDENT': {'precision': 0.1813795568263045,
  'recall': 0.7435897435897436,
  'f1': 0.29162476655652925,
  'number': 1365,
  'f5_score': 0.6643840789506812},
 'PHONE_NUM': {'precision': 0.2727272727272727,
  'recall': 1.0,
  'f1': 0.42857142857142855,
  'number': 6,
  'f5_score': 0.9069767441860465},
 'STREET_ADDRESS': {'precision': 0.0,
  'recall': 0.0,
  'f1': 0.0,
  'number': 2,
  'f5_score': 0.0},
 'URL_PERSONAL': {'precision': 0.06830818109610802,
  'recall': 0.7818181818181819,
  'f1': 0.1256391526661797,
  'number': 110,
  'f5_score': 0.557745073584435},
 'USERNAME': {'precision': 0.0,
  'recall': 0.0,
  'f1': 0.0,
  'number': 6,
  'f5_score': 0.0},
 'overall_precision': 0.16555812608444187,
 'overall_recall': 0.7129514321295